In [2]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
#Imports & NLTK Downloads
# Imports
import re
import random
import pandas as pd
from typing import List

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

import nltk

# Download NLTK data (run once)
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)

from datasets import Dataset


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
#Synthetic Dataset Generator
LABELS = ["negative", "neutral", "positive"]

lagos_locations = [
    "Lekki", "Ajah", "Ikorodu", "Surulere", "Yaba", "Victoria Island",
    "Ikeja", "Agege", "Iyana Ipaja"
]

pidgin_phrases_negative = [
    "water don full everywhere",
    "road don block finish",
    "I dey stranded for",
    "house don flood",
    "no motor fit pass",
    "we need help abeg",
    "LASEMA never show",
    "NEMA where una dey",
]

english_negative = [
    "My street is completely flooded",
    "Cars are stuck in the water",
    "People are trapped in their houses",
    "No rescue team yet",
    "The drainage has totally failed",
    "This flood is getting worse",
]

neutral_phrases = [
    "It rained heavily in",
    "Serious downpour this morning around",
    "Flood reported in parts of",
    "Traffic is slow because of water on the road in",
    "Update: water level rising in",
]

positive_phrases = [
    "LASEMA is finally on ground at",
    "Rescue teams just arrived at",
    "Water is going down gradually in",
    "Neighbours are helping each other at",
    "Government just sent trucks to",
]


def generate_synthetic_tweet(label: str) -> str:
    loc = random.choice(lagos_locations)
    
    if label == "negative":
        if random.random() < 0.5:
            base = random.choice(pidgin_phrases_negative)
            return f"{base} for {loc} o. This Lagos flood no be small thing."
        else:
            base = random.choice(english_negative)
            return f"{base} in {loc}. #LagosFlood #flood"
    
    elif label == "neutral":
        base = random.choice(neutral_phrases)
        return f"{base} {loc}. #Lagos #rain"
    
    elif label == "positive":
        base = random.choice(positive_phrases)
        return f"{base} {loc}. Thank God things are improving. 🙏"
    
    return ""


def generate_synthetic_dataset(n_per_class: int = 600) -> pd.DataFrame:
    data = []
    for label in LABELS:
        for _ in range(n_per_class):
            txt = generate_synthetic_tweet(label)
            data.append({"text": txt, "label": label})
    random.shuffle(data)
    return pd.DataFrame(data)


In [5]:
#Preprocessing & Pidgin Normalization
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

pidgin_normalization_dict = {
    "dey": "is",
    "don": "already",
    "go-slow": "traffic",
    "goslow": "traffic",
    "hold up": "traffic",
    "abeg": "please",
    "no be small thing": "very serious",
    "wahala": "problem",
    "una": "you all",
    "o": "",  # often emphasis
}


def normalize_pidgin(text: str) -> str:
    text = text.lower()
    for k, v in pidgin_normalization_dict.items():
        text = re.sub(rf"\b{k}\b", v, text)
    return text


def clean_text(text: str) -> str:
    # Remove URLs, mentions, hashtags
    text = re.sub(r"http\S+", " ", text)
    text = re.sub(r"@\w+", " ", text)
    text = re.sub(r"#", " ", text)

    # Lowercase & normalize pidgin
    text = normalize_pidgin(text)

    # Keep only letters and spaces
    text = re.sub(r"[^a-zA-Z\s]", " ", text)

    # Tokenize, remove stopwords, lemmatize
    tokens = nltk.word_tokenize(text)
    tokens = [
        lemmatizer.lemmatize(t)
        for t in tokens
        if t not in stop_words and len(t) > 2
    ]

    return " ".join(tokens)


In [6]:
#Generate Dataset & Inspect
df = generate_synthetic_dataset(n_per_class=600)  # total 1800 samples
df["clean_text"] = df["text"].apply(clean_text)

print(df.head())
print("\nLabel distribution:")
print(df["label"].value_counts())


                                                text     label  \
0  Rescue teams just arrived at Victoria Island. ...  positive   
1  The drainage has totally failed in Ajah. #Lago...  negative   
2  My street is completely flooded in Ajah. #Lago...  negative   
3  NEMA where una dey for Ikeja o. This Lagos flo...  negative   
4  LASEMA is finally on ground at Surulere. Thank...  positive   

                                          clean_text  
0  rescue team arrived victoria island thank god ...  
1      drainage totally failed ajah lagosflood flood  
2    street completely flooded ajah lagosflood flood  
3                     nema ikeja lagos flood serious  
4  lasema finally ground surulere thank god thing...  

Label distribution:
label
positive    600
negative    600
neutral     600
Name: count, dtype: int64


In [7]:
#Train & Evaluate SVM Baseline
X_train, X_test, y_train, y_test = train_test_split(
    df["clean_text"],
    df["label"],
    test_size=0.2,
    random_state=42,
    stratify=df["label"]
)

svm_pipeline = Pipeline([
    ("tfidf", TfidfVectorizer(ngram_range=(1, 2), max_features=10000)),
    ("clf", LinearSVC()),
])

svm_pipeline.fit(X_train, y_train)

y_pred_svm = svm_pipeline.predict(X_test)

print("=== SVM CLASSIFICATION REPORT ===")
print(classification_report(y_test, y_pred_svm, labels=LABELS))

print("=== SVM CONFUSION MATRIX (neg, neu, pos) ===")
print(confusion_matrix(y_test, y_pred_svm, labels=LABELS))


=== SVM CLASSIFICATION REPORT ===
              precision    recall  f1-score   support

    negative       1.00      1.00      1.00       120
     neutral       1.00      1.00      1.00       120
    positive       1.00      1.00      1.00       120

    accuracy                           1.00       360
   macro avg       1.00      1.00      1.00       360
weighted avg       1.00      1.00      1.00       360

=== SVM CONFUSION MATRIX (neg, neu, pos) ===
[[120   0   0]
 [  0 120   0]
 [  0   0 120]]


In [8]:
#Prepare Data for Transformer
# Use original text for transformer model
train_df, test_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df["label"]
)

label2id = {l: i for i, l in enumerate(LABELS)}
id2label = {i: l for l, i in label2id.items()}

from datasets import Dataset

train_ds = Dataset.from_pandas(train_df[["text", "label"]])
test_ds = Dataset.from_pandas(test_df[["text", "label"]])

def add_labels(batch):
    return {"labels": [label2id[l] for l in batch["label"]]}

train_ds = train_ds.map(add_labels, batched=True)
test_ds = test_ds.map(add_labels, batched=True)

MODEL_NAME = "bert-base-uncased"  # later you can swap to CrisisBERT-like model
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_batch(batch):
    return tokenizer(
        batch["text"],
        padding="max_length",
        truncation=True,
        max_length=64
    )

train_ds = train_ds.map(tokenize_batch, batched=True)
test_ds = test_ds.map(tokenize_batch, batched=True)

train_ds.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "labels"]
)
test_ds.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "labels"]
)


Map:   0%|          | 0/1440 [00:00<?, ? examples/s]

Map:   0%|          | 0/360 [00:00<?, ? examples/s]

Map:   0%|          | 0/1440 [00:00<?, ? examples/s]

Map:   0%|          | 0/360 [00:00<?, ? examples/s]

In [9]:
#Define & Train Transformer Model
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(LABELS),
    id2label=id2label,
    label2id=label2id
)

training_args = TrainingArguments(
    output_dir="./bert_lagos_floods",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=20,
    save_total_limit=2,
    load_best_model_at_end=False
)

from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(pred):
    logits, labels = pred
    preds = logits.argmax(axis=-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average="weighted"
    )
    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\USER\AppData\Local\Temp\ipykernel_11524\3351515455.py:38: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
C:\Users\USER\anaconda3\envs\crisis-nlp\lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
20,0.670900
40,0.184000
60,0.041500
80,0.014400
100,0.008400
120,0.005900
140,0.004600
160,0.003900
180,0.003600
200,0.003100


TrainOutput(global_step=270, training_loss=0.07039890529380904, metrics={'train_runtime': 1904.7695, 'train_samples_per_second': 2.268, 'train_steps_per_second': 0.142, 'total_flos': 142081245573120.0, 'train_loss': 0.07039890529380904, 'epoch': 3.0})

In [10]:
#Evaluate Transformer Model
metrics = trainer.evaluate()
print("=== TRANSFORMER AGGREGATE METRICS ===")
print(metrics)

preds_output = trainer.predict(test_ds)
y_true = preds_output.label_ids
y_pred = preds_output.predictions.argmax(axis=-1)

print("\n=== TRANSFORMER CLASSIFICATION REPORT ===")
print(classification_report(
    [id2label[i] for i in y_true],
    [id2label[i] for i in y_pred],
    labels=LABELS
))

print("\n=== TRANSFORMER CONFUSION MATRIX (neg, neu, pos) ===")
print(confusion_matrix(
    [id2label[i] for i in y_true],
    [id2label[i] for i in y_pred],
    labels=LABELS
))


C:\Users\USER\anaconda3\envs\crisis-nlp\lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


=== TRANSFORMER AGGREGATE METRICS ===
{'eval_loss': 0.0018760310485959053, 'eval_accuracy': 1.0, 'eval_precision': 1.0, 'eval_recall': 1.0, 'eval_f1': 1.0, 'eval_runtime': 38.087, 'eval_samples_per_second': 9.452, 'eval_steps_per_second': 0.315, 'epoch': 3.0}


C:\Users\USER\anaconda3\envs\crisis-nlp\lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)



=== TRANSFORMER CLASSIFICATION REPORT ===
              precision    recall  f1-score   support

    negative       1.00      1.00      1.00       120
     neutral       1.00      1.00      1.00       120
    positive       1.00      1.00      1.00       120

    accuracy                           1.00       360
   macro avg       1.00      1.00      1.00       360
weighted avg       1.00      1.00      1.00       360


=== TRANSFORMER CONFUSION MATRIX (neg, neu, pos) ===
[[120   0   0]
 [  0 120   0]
 [  0   0 120]]


In [11]:
import joblib

# 1. Save the SVM pipeline
joblib.dump(svm_pipeline, "svm_pipeline.pkl")
print("Saved svm_pipeline.pkl")

# 2. Save the transformer model + tokenizer
model_dir = "./bert_lagos_model"
tokenizer.save_pretrained(model_dir)
model.save_pretrained(model_dir)
print("Saved transformer model + tokenizer to bert_lagos_model/")

# 3. Save the synthetic dataset (optional but useful for dashboard charts)
df.to_csv("synthetic_lagos_floods.csv", index=False)
print("Saved synthetic_lagos_floods.csv")


Saved svm_pipeline.pkl
Saved transformer model + tokenizer to bert_lagos_model/
Saved synthetic_lagos_floods.csv
